In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# 11.3 날짜 범위, 빈도, 이동

In [2]:
# pandas에서 일반적인 시계열은 불규칙적인 것으로 간주된다. 즉, 고정된 빈도를 갖지 않는다.
# 대부분의 애플리케이션에서 이는 충분하다. 하지만 시계열 안에서 누락된 값이 발생할지라도 일별, 월별 혹은 매 15분 같은 상대적인 고정 빈도에서의 작업이 요구되는 경우도 있다.

In [3]:
# 다행스럽게도 pandas에는 리샘플링, 표준 시계열 빈도 모음, 빈도 추론 그리고 고정된 빈도의 날짜 범위를 위한 도구가 있다. 
# 예를 들어 아래 예제 시계열을 고정된 일 빈도로 변환하려면 resample 메서드를 사용하면 된다. 

In [4]:
from datetime import datetime
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.standard_normal(6), index=dates)
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [5]:
resampler = ts.resample("D")

In [6]:
# 문자열 "D"는 일 빈도로 해석된다. 
# 빈도 간 변환이나 리샘플링은 큰 주제이므로 다음에 따로 다루도록 하겠다(11.6절 "리샘플링과 빈도 변환").
# 여기에서는 기본 빈도와 다중 빈도를 어떻게 사용하는지 살펴보도록 하자. 

# 11.3.1 날짜 범위 생성하기

In [7]:
# 앞에서는 설명 없이 그냥 사용했지만 pandas.date_range를 사용하면 특정 빈도에 따라 지정한 길이만큼의 DatetimeIndex를 생성한다는 사실을 눈치 챘을 것이다.

In [8]:
index = pd.date_range("2012-04-01", "2012-06-01")

In [9]:
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [11]:
# 기본적으로 date_range는 일별 타임스탬프를 생성한다. 만약 시작 날짜나 종료 날짜만 넘긴다면 생성할 기간의 숫자를 함께 전달해야 한다.

In [12]:
pd.date_range(start="2012-04-01", periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [13]:
pd.date_range(end="2012-06-01", periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [16]:
# 시작과 종료 날짜는 생성된 날짜 색인에 대해 엄격한 경계를 정의한다.  
# 예를 들어 날짜 색인이 각 월의 마지막 영업일을 포함하도록 하고 싶다면 빈도값으로 "BM"(월 영업마감일.[표 11-4])을 전달할 것이다. 페이지 438
# 그러면 이 기간 안에 들어오는 날짜들만 포함된다. 

In [17]:
pd.date_range("2000-01-01", "2000-12-01", freq="BM")

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [18]:
# date_range는 기본적으로 시작 시간이나 종료 시간의 타임스탬프(존재한다면)를 보존한다. 

In [19]:
pd.date_range("2012-05-02 12:56:31", periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [20]:
# 가끔은 시간 정보를 포함하여 시작 날짜와 종료 날짜를 갖고 있으나 관례에 따라 자정에 맞추어 타임스탬프를 정규화하고 싶을 때가 있다.
# 이렇게 하려면 normalize 옵션을 사용한다. 

In [21]:
pd.date_range("2012-05-02 12:56:31", periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

# 11.3.2 빈도와 날짜 오프셋

In [22]:
# pandas에서 빈도는 기본 빈도와 배수의 조합으로 이루어진다. 기본 빈도는 보통 "M(월별)", "H(시간별)" 처럼 짧은 문자열로 참조된다. 
# 각 기본 빈도에는 일반적으로 날짜 오프셋이라고 불리는 객체를 사용할 수 있다. 
# 예를 들어 시간별 빈도는 Hour 클래스를 사용해서 표현할 수 있다. 

In [23]:
from pandas.tseries.offsets import Hour, Minute

In [24]:
hour = Hour()

In [25]:
hour

<Hour>

In [26]:
# 이 오프셋의 곱은 정수를 넘겨서 구할 수 있다. 

In [27]:
four_hours = Hour(4)

In [28]:
four_hours

<4 * Hours>

In [29]:
# 대부분의 애플리케이션에서는 이런 객체들을 직접 만들어야할 경우는 절대 없겠지만 대신 "H" 또는 "4H"처럼 문자열로 표현하게 될 것이다. 
# 기본 빈도 앞에 정수를 두면 해당 빈도의 곱을 생성한다. 

In [30]:
pd.date_range("2000-01-01", "2000-01-03 23:59", freq="4h")

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [31]:
# 여러 오프셋을 덧셈으로 합칠 수 있다.

In [32]:
Hour(2) + Minute(30)

<150 * Minutes>

In [33]:
# 유사하게 빈도 문자열로 "1h30min"을 넘겨도 같은 표현으로 잘 해석된다. 

In [34]:
pd.date_range("2000-01-01", periods=10, freq="1h30min")

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

In [35]:
# 어떤 빈도는 시간 상에서 균일하게 자리 잡고 있지 않은 경우도 있다.
# 예를 들어 "M"(월 마지막 일)은 월중 일수에 의존적이며 "BM"(월 영업마감일)은 월말이 주말인지 아닌지에 따라 다르다.
# 이를 표현할 수 있는 적당한 용어가 없어서 우선 앵커드 오프셋이라고 부른다.
# [표 11-4]에 pandas에서 사용 가능한 빈도 코드와 날짜 오프셋 클래스를 정리해두었다.

- NOTE_ pandas에 없는 날짜 연산을 제공하기 위해 사용자가 직접 사용자 빈도 클래스를 정의할 수 있지만, 그 내용은 이 책에서 다룰 만한 내용이 아니므로 제외했다.

In [36]:
# 월별 주차
# 한 가지 유용한 빈도 클래스는 WOM으로 시작하는 "월별 주차"다. 월별 주차를 사용하면 매월 셋째 주 금요일 같은 날짜를 얻을 수 있다. 

In [37]:
rng = pd.date_range("2012-01-01", "2012-09-01", freq="WOM-3FRI")

In [38]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

# 11.3.3 데이터 시프트

In [39]:
# 시프트는 데이터를 시간 축에서 앞이나 뒤로 이동하는 것을 의미한다. 
# Series와 DataFrame은 색인은 변경하지 않고 데이터를 앞이나 뒤로 느슨한 시프트를 수행하는 shift 메서드를 가지고 있다. 

In [40]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range("1/1/2000", periods=4, freq="M"))

In [41]:
ts

2000-01-31    0.092908
2000-02-29    0.281746
2000-03-31    0.769023
2000-04-30    1.246435
Freq: M, dtype: float64

In [42]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.092908
2000-04-30    0.281746
Freq: M, dtype: float64

In [43]:
ts.shift(-2)

2000-01-31    0.769023
2000-02-29    1.246435
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

In [45]:
# 이렇게 시프트를 하게 되면 시계열의 시작이나 끝에 결측치가 발생하게 된다.

In [46]:
# shift는 일반적으로 한 시계열 내에서, 혹은 DataFrame의 컬럼으로 표현할 수 있는 여러 시계열에서의 퍼센트 변화를 계산할 때 흔히 사용되며, 코드로는 다음과 같이 표현한다. 

In [47]:
ts / ts.shift(1) - 1

2000-01-31         NaN
2000-02-29    2.032532
2000-03-31    1.729487
2000-04-30    0.620804
Freq: M, dtype: float64

In [48]:
# 느슨한 시프트는 색인을 바꾸지 않기 때문에 어떤 데이터는 버려지기도 한다. 
# 그래서 만약 빈도를 알고 있다면 shift에 빈도를 넘겨서 타임스탬프가 확장되도록 할 수 있다. 

In [49]:
ts.shift(2, freq="M")

2000-03-31    0.092908
2000-04-30    0.281746
2000-05-31    0.769023
2000-06-30    1.246435
Freq: M, dtype: float64

In [50]:
# 다른 빈도를 넘겨도 되는데, 이를 통해 아주 유연하게 데이터를 밀거나 당기는 작업을 할 수 있다.

In [51]:
ts.shift(3, freq="D")

2000-02-03    0.092908
2000-03-03    0.281746
2000-04-03    0.769023
2000-05-03    1.246435
dtype: float64

In [52]:
ts.shift(1, freq="90T")

2000-01-31 01:30:00    0.092908
2000-02-29 01:30:00    0.281746
2000-03-31 01:30:00    0.769023
2000-04-30 01:30:00    1.246435
dtype: float64

In [53]:
# 여기서 T는 분을 나타낸다. 

- 오프셋만큼 날짜 시프트하기

In [54]:
# pandas의 날짜 오프셋은 datetime이나 Timestamp 객체에서도 사용할 수 있다.

In [55]:
from pandas.tseries.offsets import Day, MonthEnd

In [56]:
now = datetime(2011, 11, 17)

In [57]:
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [58]:
# 만일 MonthEnd 같은 앵커드 오프셋을 추가한다면 빈도 규칙의 다음 날짜로 롤 포워드된다. 

In [59]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [60]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

In [61]:
# 앵커드 오프셋은 rollforward와 rollback 메서드를 사용해서 명시적으로 각각 날짜를 앞으로 밀거나 뒤로 당길 수 있다. 

In [62]:
offset = MonthEnd()

In [63]:
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [64]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

In [65]:
# 이 메서드를 groupby와 함께 사용하면 날짜 오프셋을 영리하게 사용할 수 있다. 

In [66]:
ts = pd.Series(np.random.randn(20),
               index=pd.date_range("1/15/2000", periods=20, freq="4d"))

In [67]:
ts

2000-01-15    1.007189
2000-01-19   -1.296221
2000-01-23    0.274992
2000-01-27    0.228913
2000-01-31    1.352917
2000-02-04    0.886429
2000-02-08   -2.001637
2000-02-12   -0.371843
2000-02-16    1.669025
2000-02-20   -0.438570
2000-02-24   -0.539741
2000-02-28    0.476985
2000-03-03    3.248944
2000-03-07   -1.021228
2000-03-11   -0.577087
2000-03-15    0.124121
2000-03-19    0.302614
2000-03-23    0.523772
2000-03-27    0.000940
2000-03-31    1.343810
Freq: 4D, dtype: float64

In [68]:
ts.groupby(offset.rollforward).mean()

2000-01-31    0.313558
2000-02-29   -0.045622
2000-03-31    0.493236
dtype: float64

In [69]:
# 물론 가장 쉽고 빠른 방법은 resample을 사용하는 것이다. 

In [70]:
ts.resample("M").mean()

2000-01-31    0.313558
2000-02-29   -0.045622
2000-03-31    0.493236
Freq: M, dtype: float64